In [ ]:
import json

import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.metrics import f1_score

In [ ]:
with open("../resources/datasets/combat.json") as f:
    ds = json.load(f)
ds

In [ ]:
records = []
for combat in ds:
    army1 = combat["army1"]
    army2 = combat["army2"]
    records.append(
        dict(
            # health1=sum(u["health"] + u["shield"] for u in army1),
            # health2=sum(u["health"] + u["shield"] for u in army2),
            # dps1=sum(u["ground_dps"] for u in army1) / len(army1),
            # dps2=sum(u["ground_dps"] for u in army2) / len(army2),
            strength1=sum(u["ground_dps"] for u in army1) / len(army1),
            strength2=sum(u["ground_dps"] for u in army2) / len(army2),
            size1=sum(u["health"] + u["shield"] for u in army1),
            size2=sum(u["health"] + u["shield"] for u in army2),
            winner=combat["result"] > 0,
        )
    )
df = pd.DataFrame.from_records(records)
df["lancester1_linear"] = df["strength1"] * df["size1"]
df["lancester2_linear"] = df["strength2"] * df["size2"]
df["lancester_linear"] = df["lancester1_linear"] > df["lancester2_linear"]
df["lancester1_square"] = df["strength1"] * df["size1"] ** 2
df["lancester2_square"] = df["strength2"] * df["size2"] ** 2
df["lancester_square"] = df["lancester1_square"] > df["lancester2_square"]
df["lancester1_15"] = df["strength1"] * df["size1"] ** (1.5)
df["lancester2_15"] = df["strength2"] * df["size2"] ** (1.5)
df["lancester_15"] = df["lancester1_15"] > df["lancester2_15"]

In [ ]:
f1_score(df["winner"], df["lancester_linear"])

In [ ]:
f1_score(df["winner"], df["lancester_square"])

In [ ]:
f1_score(df["winner"], df["lancester_15"])

In [ ]:
powers = np.linspace(1, 2, 100)
ratios = [(df["strength1"] * df["size1"] ** p > df["strength2"] * df["size2"] ** p) for p in powers]
scores = [f1_score(df["winner"], r) for r in ratios]
px.line(x=powers, y=scores)

In [ ]:
px.scatter(
    df,
    x="lancester1_linear",
    y="lancester2_linear",
    color="winner",
)